In [12]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [13]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Test/inpatient_specialty_test.csv').iloc[:,1:]

In [14]:
inpatient_specialty

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
0,100004,81.261813,2014-04-06 04:37:21.0,2014-04-08 06:17:29.0,CARDIAC-STEP DOWN UNIT,Florida
1,100021,61.126677,2011-03-29 19:12:46.0,2011-03-31 21:53:53.0,PTSD RESIDENTIAL REHAB PROG,Colorado
2,100035,52.316254,2014-07-13 14:10:34.0,2014-07-15 16:10:05.0,BLIND REHAB OBSERVATION,District of Columbia
3,100041,89.583528,2014-11-05 18:42:42.0,2014-11-06 04:08:18.0,NH SHORT-STAY CONTINUING CARE,Minnesota
4,100041,89.595286,2014-11-10 01:51:04.0,2014-11-10 01:51:04.0,ORTHOPEDIC,Minnesota
...,...,...,...,...,...,...
386891,9996,77.010416,2005-12-21 07:22:13.0,2005-12-22 03:51:27.0,GEM REHABILITATION MEDICINE,Pennsylvania
386892,9996,84.296993,2013-04-06 13:13:12.0,2013-04-08 17:12:56.0,GEM INTERMEDIATE CARE,Pennsylvania
386893,99964,73.451708,2016-07-30 14:22:55.0,2016-07-30 14:22:55.0,GENERAL(ACUTE MEDICINE),Oregon
386894,99989,71.251809,2003-10-03 12:52:03.0,2003-10-04 04:18:52.0,DOD BEDS IN VA FACILITY,Ohio


In [15]:
pd.isnull(inpatient_specialty).sum() 

Internalpatientid         0
Age at specialty          0
Specialty start date      0
Specialty end date      128
Specialty                 0
State                     0
dtype: int64

In [16]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
747,100483,79.373001,2024-01-07 19:57:49.0,NaN,GENERAL CWT/TR,California
843,101138,63.782778,2019-10-29 14:31:54.0,NaN,NH LONG STAY DEMENTIA CARE,Massachusetts
1102,102906,74.987656,2023-07-28 23:43:03.0,NaN,PODIATRY,Nevada
4956,109712,48.949437,2021-05-23 00:27:39.0,NaN,REHABILITATION MEDICINE,Pennsylvania
6696,105831,73.976405,2021-12-15 09:01:29.0,NaN,VASCULAR,Kentucky
...,...,...,...,...,...,...
371889,74697,71.815637,2024-05-27 14:45:55.0,NaN,SUBSTANCE ABUSE RES TRMT PROG,Kentucky
375418,74288,97.356966,2023-07-03 23:18:36.0,NaN,NH SHORT STAY SKILLED NURSING,Pennsylvania
375554,80003,79.472534,2023-11-09 05:04:14.0,NaN,LONG TERM PSYCHIATRY(>45 DAYS),Ohio
383120,91831,84.030945,2021-11-12 07:26:56.0,NaN,"PULMONARY, TUBERCULOSIS",Michigan


# Preprocess Time
Calculate period mean and std values.

In [17]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]

/tmp/ipykernel_264811/1186491515.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]


,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State


In [18]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [19]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

/tmp/ipykernel_264811/2218735275.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))


In [20]:
period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [21]:
period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

# Preprocess Specialty
Then consider the specialty, since it has a lot of different values, we want to simplify it in to several classes.
By studying the values, we can have the classify it into Medical Specialties (encoded as 0) and Rehabilitation and Support Programs (encoded as 1). And finally impute the no specific value (3.75%) with imputate_nan_binary.

In [22]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [23]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [24]:
specialty_count = inpatient_specialty.groupby('Internalpatientid')['Specialty'].count().reset_index()

In [25]:
specialty_count = specialty_count.rename(columns={'Specialty': 'specialty count'})

In [26]:
specialty_medical_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(0).sum()).reset_index()

In [27]:
specialty_support_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(1).sum()).reset_index()

In [28]:
specialty_medical_count = specialty_medical_count.rename(columns={'Specialty encoded': 'specialty medical count'})
specialty_support_count = specialty_support_count.rename(columns={'Specialty encoded': 'specialty support count'})

In [29]:
pd.isnull(period_std).sum() 

Internalpatientid    0
period std           2
dtype: int64

In [30]:
period_std[pd.isnull(period_std['period std'])]

,Internalpatientid,period std
12880,98420,NaN
15195,115672,NaN


In [31]:
inpatient_specialty[inpatient_specialty['Internalpatientid'] == 35865]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded


In [32]:
inpatient_specialty[inpatient_specialty['Internalpatientid'] == 162785]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded


In [33]:
period_mean[pd.isnull(period_mean['period mean'])]

,Internalpatientid,period mean
12880,98420,NaN
15195,115672,NaN


# Preprocess Age

In [34]:
age_mean = inpatient_specialty.groupby('Internalpatientid')['Age at specialty'].mean().reset_index()
age_mean = age_mean.rename(columns={'Age at specialty': 'mean age at specailty'})

# Merge All

In [35]:
listTables = [period_mean, period_std, specialty_medical_count,specialty_support_count, specialty_count ]
inpatient_specialty_preprocessed = age_mean.copy()
for i in listTables:
    inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.merge(i,how = 'left', on = 'Internalpatientid')


In [36]:
inpatient_specialty_preprocessed

,Internalpatientid,mean age at specailty,period mean,period std,specialty medical count,specialty support count,specialty count
0,7,62.879044,3.579801,7.603465,9,4,13
1,9,51.778496,1.945340,2.069379,7,6,13
2,12,72.448082,2.031376,2.952513,12,0,12
3,17,80.987588,1.622804,2.126398,13,18,31
4,22,58.983378,1.466395,2.058525,3,6,9
...,...,...,...,...,...,...,...
22250,168995,68.384928,0.668522,0.750610,4,3,7
22251,169011,64.960772,1.668035,2.455277,2,1,4
22252,169037,84.503238,11.138164,19.410162,4,2,6
22253,169059,79.918093,4.455550,4.455550,1,1,2


In [37]:
inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.dropna(subset=['period mean'])

In [38]:
inpatient_specialty_preprocessed.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_preprocessed_test.csv')

In [39]:
pd.isnull(inpatient_specialty_preprocessed).sum()

Internalpatientid          0
mean age at specailty      0
period mean                0
period std                 0
specialty medical count    0
specialty support count    0
specialty count            0
dtype: int64